In [ ]:
from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel
import base64
from openai import OpenAI

# Initialize OpenAI client
OPENAI_API_KEY = "your_openai_api_key_here"
client = OpenAI(api_key=OPENAI_API_KEY)

# Initialize FastAPI app
app = FastAPI()

# Function to encode image to Base64
def encode_image(file_path):
    with open(file_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Function to analyze image using OpenAI
def analyze_image_with_openai(file_path):
    try:
        base64_image = encode_image(file_path)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "分析圖片上面有甚麼樣的路牌標示"},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64_image}",
                            },
                        },
                    ],
                }
            ],
            max_tokens=500,
        )
        ans = response.choices[0]
        ans_msg = ans.message.content
        return ans_msg
    except Exception as e:
        return f"Image Analysis Error: {e}"

# API endpoint to upload and analyze image
@app.post("/analyze-image/")
async def analyze_image(file: UploadFile = File(...)):
    try:
        file_path = f"temp_{file.filename}"

        # Save uploaded file
        with open(file_path, "wb") as f:
            content = await file.read()
            f.write(content)

        # Analyze the image
        result = analyze_image_with_openai(file_path)

        # Clean up temporary file
        import os
        os.remove(file_path)

        return {"result": result}
    except Exception as e:
        return {"error": str(e)}

# Run with: uvicorn filename:app --reload
